In [1]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Install Libraries
%%capture
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install accelerate

In [4]:
import os
import torch
import pandas as pd
import sys
import glob
import numpy as np
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [6]:
# set data paths
device = 'cuda' if torch.cuda.is_available() else 'cpu'

root_dir = "gdrive/My Drive/Master_Thesis/"
model_dir = os.path.join(root_dir, 'models/sexism_classifiers')
model_name = "cardiffnlp/tweet-topic-21-multi"
model_path = os.path.join(model_dir, model_name)
model_path = os.path.join(model_path, "09,06,2023--21,45")

sample_prediction = os.path.join(root_dir, 'predictions/Base/gpt2-medium_16,05,2023-21,09_Base_06,06,2023--20,42.csv')

In [7]:
data_path = os.path.join(root_dir, "data/Custom/T8-S10.csv")

In [9]:
df = pd.read_csv(data_path)

In [11]:
df["Prediction"] = df["Counter_Speech"]

In [13]:
save_dir = os.path.join(root_dir, 'predictions/Small/')
file_name = "Human.csv"


os.makedirs(save_dir, exist_ok=True)  
df.to_csv(save_dir + file_name)  

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)

In [ ]:
test_raw = pd.read_csv(sample_prediction)
df_test = test_raw.copy()

def format_multiclass(item):
  return item.split("/")

def format_multiclass_2(item):
  if item != item:
    return []
  return item.upper().split(",")

df_test["Target"] = np.where(df_test["Target"] == "Islamophobia", "MUSLIMS", df_test["Target"])
df_test["Target"] = df_test["Target"].map(format_multiclass)
df_test["Target_2"] = df_test["Target_2"].map(format_multiclass_2)
df_test["Target"] = df_test["Target"] + df_test["Target_2"]


types = [None]*len(model.config.label2id)
for label, idx in model.config.label2id.items():
  types[int(idx)] = label

tuple_head=[]
for index,row in df_test.iterrows():
  label = []
  tuple_temp=[row['Hate_Speech']]
  tuple_temp.append(row["Counter_Speech"])
  tuple_temp.append(row["Prediction"])
  tuple_temp.append(row["Target"])
  for target in types:
    if target in row["Target"]:
      label.append(1)
    else:
      label.append(0)
  tuple_temp.append(label)
  tuple_head.append(tuple_temp)

df=pd.DataFrame(tuple_head,columns=['Hate_Speech',"Counter_Speech","Prediction","Target","labels"])

In [ ]:
from tqdm import tqdm
import statistics

def get_label_int(item):
  return int(model.config.label2id[item])

def argument_type(prediction):
  print('-'*80)
  print('Calculating Argument Type of predictions')
  
  results = []
  bs = 8

  # prepare the input
  inputs = prediction
  # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
  sigmoid = torch.nn.Sigmoid()

  # Batch Evaluation
  for i in tqdm(range(0, len(inputs), bs)):
    batch = tokenizer(
        inputs[i:i + bs],
        return_tensors='pt',
        padding=True).to(model.device)
        
    with torch.inference_mode():
      logits = model(**batch).logits
      probs = sigmoid(torch.Tensor(logits))

      # next, use threshold to turn them into integer predictions
      y_pred = np.zeros(probs.shape)
      y_pred[np.where(probs.cpu() >= 0.7)] = 1

    results = results + y_pred.tolist()

  return results

In [ ]:
predictions = df['Prediction'].tolist()
predicted_labels = argument_type(predictions)

--------------------------------------------------------------------------------
Calculating Argument Type of predictions


100%|██████████| 299/299 [00:17<00:00, 17.50it/s]


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

labels=df["labels"].tolist()

f1_micro_average = f1_score(y_true=labels, y_pred=predicted_labels, average='micro')
roc_auc = roc_auc_score(labels, predicted_labels, average = 'micro')
accuracy = accuracy_score(labels, predicted_labels)
# return as dictionary
metrics = {'f1': f1_micro_average,
           'roc_auc': roc_auc,
           'accuracy': accuracy}

metrics 

{'f1': 0.8937421251574968,
 'roc_auc': 0.9349320357232037,
 'accuracy': 0.8822296730930428}

# -----------------

In [ ]:
text = "Women are human beings with rights. They have the right to be treated with respect and dignity."
inputs = tokenizer(text, return_tensors="pt").to(model.device)
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[str(predicted_class_id)]